In [1]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Step 2: Load and Prepare Data
data_1 = pd.read_csv(r'C:\Users\bravo\OneDrive\OneDrive Files\Desktop\train_set_1.csv')
data_2 = pd.read_csv(r'C:\Users\bravo\OneDrive\OneDrive Files\Desktop\train_set_2.csv')
data_3 = pd.read_csv(r'C:\Users\bravo\OneDrive\OneDrive Files\Desktop\train_set_3.csv')

# Step 3: Generate Features for Financial Time Series Data
def generate_features(data):
    lag = 5
    data['SMA_5'] = data['value'].rolling(window=5).mean()
    data['SMA_20'] = data['value'].rolling(window=20).mean()

    for i in range(1, lag + 1):
        data[f'Lag_{i}'] = data['value'].shift(i)
    
    data['Rolling_STD_5'] = data['value'].rolling(window=5).std()
    data['Rolling_STD_20'] = data['value'].rolling(window=20).std()
    
    roc_period = 1
    data['ROC'] = (data['value'].diff(roc_period).shift(-1) > 0).astype(int)  # Shift ROC as required

    return data

data_1 = generate_features(data_1)
data_2 = generate_features(data_2)
data_3 = generate_features(data_3)

# Step 4: Prepare Features and Labels for all Datasets
def prepare_data(data):
    lag = 5
    data = data.dropna()
    
    X = data[['SMA_5', 'SMA_20', 'Rolling_STD_5', 'Rolling_STD_20'] + [f'Lag_{i}' for i in range(1, lag + 1)]]
    y = data['ROC']

    return X, y

X_1, y_1 = prepare_data(data_1)
X_2, y_2 = prepare_data(data_2)
X_3, y_3 = prepare_data(data_3)

# Step 5: Split Data into Training and Test Sets for all Datasets
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test

# Step 6: Train and Evaluate Models for all Datasets
def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(32, input_shape=input_shape, activation='relu', return_sequences=True))
    model.add(LSTM(32, activation='relu', return_sequences=True))
    model.add(SimpleRNN(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_and_evaluate_rnn_model(X_train, y_train, X_test, y_test):
    early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
    input_shape = (X_train.shape[1], 1)  # Adjust the input shape for RNN
    model = create_rnn_model(input_shape)
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    print("Test Accuracy:", accuracy)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("Evaluation for Dataset 1:")
X_train_1, X_test_1, y_train_1, y_test_1 = split_data(X_1, y_1)
train_and_evaluate_rnn_model(X_train_1, y_train_1, X_test_1, y_test_1)

print("Evaluation for Dataset 2:")
X_train_2, X_test_2, y_train_2, y_test_2 = split_data(X_2, y_2)
train_and_evaluate_rnn_model(X_train_2, y_train_2, X_test_2, y_test_2)

print("Evaluation for Dataset 3:")
X_train_3, X_test_3, y_train_3, y_test_3 = split_data(X_3, y_3)
train_and_evaluate_rnn_model(X_train_3, y_train_3, X_test_3, y_test_3)


C:\Users\bravo\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Evaluation for Dataset 1:
125/125 [==============================] - 1s 2ms/step
Test Accuracy: 0.5298974230673005
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.29      0.39      2017
           1       0.52      0.77      0.62      1980

    accuracy                           0.53      3997
   macro avg       0.54      0.53      0.50      3997
weighted avg       0.54      0.53      0.50      3997

Evaluation for Dataset 2:
125/125 [==============================] - 1s 3ms/step
Test Accuracy: 0.4938704028021016
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2023
           1       0.49      1.00      0.66      1974

    accuracy                           0.49      3997
   macro avg       0.25      0.50      0.33      3997
weighted avg       0.24      0.49      0.33      3997

Evaluation for Dataset 3:


C:\Users\bravo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bravo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bravo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


125/125 [==============================] - 1s 2ms/step
Test Accuracy: 0.514635976982737
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.09      0.15      1981
           1       0.51      0.94      0.66      2016

    accuracy                           0.51      3997
   macro avg       0.54      0.51      0.40      3997
weighted avg       0.54      0.51      0.41      3997

